In [1]:
# 匯入套件
# 操作 browser 的 API
from selenium import webdriver
# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException
# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait
# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC
# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By
# 強制等待 (執行期間休息一下)
from time import sleep
# 整理 json 使用的工具
import json
# 執行 command 的時候用的
import os
import csv
import requests 
import pandas as pd
import folium

In [2]:
response_json = requests.get('https://www.camacafe.com/store_location/getList?page=1&city=&district=').json()
result = pd.DataFrame.from_dict(response_json['aaData'])
# response_json
# print(result)
result

#電話資料清洗
tel1=result['tel'].apply(pd.Series)
tel1['newnumber']=(tel1['area_code']+'-'+tel1['number'])
tel1=tel1.drop(['area_code','number','ext'], axis=1)
tel1['store_name']=result['store_name']

final = pd.merge(result, tel1)

#刪除不要的欄位資料
final = final.drop(['feature_image','tel'], axis=1)

#gps資料清洗
gps=result['location'].apply(pd.Series)

gps['store_name']=result['store_name']
# gps['latitude'] = gps['latitude'].astype(str)
# gps['longitude'] = gps['longitude'].astype(str)
gps=gps.rename(columns={'latitude':'rel-store-lat','longitude':'rel-store-lng'})

final = pd.merge(final, gps)
final = final.drop(['location'], axis=1)

#變更欄位順序
final = final[['store_name','address','content','newnumber','rel-store-lat','rel-store-lng']]

#欄位名稱更名
final=final.rename(columns={'store_name':'storeName','address':'storeAdd','content':'hours','newnumber':'tel',})
final

,storeName,storeAdd,hours,tel,rel-store-lat,rel-store-lng
0,台北和平店,臺北市大安區和平東路三段6號,<p>週一～週五 7:30 ～ 20:00 &nbsp;例假日 9:00 ～ 18:00</p>,02-2739-0918,25.024509,121.546950
1,台北長安店,臺北市中山區長安東路二段94號,週一～週五 7:30 ～ 19:00 例假日 9:00 ～ 18:00,02-2518-5377,25.048333,121.534617
2,台北六張犁,臺北市信義區基隆路二段195號,週一～週五 7:30 ～ 18:00 例假日 9:00 ～ 18:00,02-2377-0168,25.024741,121.553212
3,台北南港店,臺北市南港區南港路一段153號,週一～週五 7:30 ～ 19:00 例假日 9:00 ～ 18:00,02-2788-5520,25.055044,121.613650
4,高雄明華店,高雄市左營區明華一路205號,週一～週五 7:30 ～ 20:00 例假日 9:00 ～ 19:00,07-558-7188,22.662795,120.306143
...,...,...,...,...,...,...
123,南京龍江店(亮點),臺北市中山區南京東路三段82 號,<p>平日: 0730-2000 假日: 0800-1800</p>,02-2500-0592,25.051799,121.539317
124,台北天津店(亮點),臺北市中山區長安東路一段15號一樓,<p>營業時間: 週一～週五 7:30 ～ 20:00&nbsp; 例假日 9:00 ～ 1...,02-25713300,25.049787,121.522686
125,北屯昌平店(即將開幕),臺中市北屯區昌平路二段8號,None,04-2241-1595,24.180111,120.690029
126,高雄鹽埕店(即將開幕),高雄市鹽埕區五福四路162號,None,07-521-1737,22.623477,120.284557


In [ ]:
#將表格輸出另存csv檔
final.to_csv('camalist.csv', mode='a+',encoding='utf-8_sig',header=True)

In [3]:
import folium
# 中文店名有亂碼問題，在這裡進行轉碼
def parse_zhch(s):
    return str(str(s).encode('ascii' , 'xmlcharrefreplace'))[2:-1]

# 定義繪製地圖中心位置和縮放大小
m = folium.Map((23.973875, 120.982024),zoom_start=7)

# 用迴圈把前面做好的店面DataFrame資料帶進來
i = -1
while i <=125:
    i = i + 1
    mrt = (final.iloc[i,4],final.iloc[i,5])
    name = final.iloc[i,0]
    folium.Circle(
        location=mrt,
        radius=100,
        popup=parse_zhch(name),
        color='red',
        fill=False,
        fill_color='red',
        fill_opacity=0.05,
    ).add_to(m)
m